In [1]:
import astropy.units as u

import gw_signal_tools

/home/user/miniconda3/envs/gw_signal_tools_light/lib/python3.11/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


# Managing Units

To get convenient displaying of units, either add unit to astropy/custom units from gw_signal_tools or define your own unit system (which just requires to specify bases, where base units are chosen and things are reduced to them by default; if some unit with prefix is also defined, reduction seems to be happending to this one)

In [2]:
from gw_signal_tools import preferred_unit_system

print(preferred_unit_system)


gw_signal_tools.preferred_unit_system = u.si

from gw_signal_tools import preferred_unit_system

print(preferred_unit_system)

<module 'gw_signal_tools.units' from '/home/user/Documents/master/gw-signal-tools/gw_signal_tools/units.py'>
<module 'astropy.units.si' from '/home/user/miniconda3/envs/gw_signal_tools_light/lib/python3.11/site-packages/astropy/units/si.py'>


First solution is better in case you want to specify custom input unit to some argument, for example milli-sunmasses. In that case, just add unit with this prefix to existing gw_signal_tools unit system and output of derivative for instance will be displayed in this unit as well

Note that derivative and thus Fisher values are always given in input units! If you want to change this, either change input unit or convert output to Quantity yourself and then separate value, unit again (in case of Fisher, this conversion would have to happen element-wise)

In [3]:
from gw_signal_tools import units


print(u.Unit(1e-3*u.Msun).to_system(units))
print(u.Unit(1e-3*u.Msun).compose(units=units))


units.def_unit(
    ['mMsun'],
    1e-3 * u.astrophys.Msun,
    namespace=units._ns
    # Optional other arguments to def_unit, like prefixes or doc
)

print(u.Unit(1e-3*u.Msun).to_system(units))
print(u.Unit(1e-3*u.Msun).compose(units=units))

[Unit("0.001 solMass")]
[Unit("0.001 solMass")]
[Unit("0.001 solMass"), Unit("mMsun")]
[Unit("mMsun"), Unit("0.001 solMass")]


Voila, there it is.

Note/tip: compose always shows the way we want things to be displayed first, so this is typically best choice because we can select first element and rely on getting best result